In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import timeit

In [4]:
mask = np.zeros((3,), dtype=bool)
mask[0] = True
mask[2] = True
A=np.zeros((3,4), dtype=bool)
A[mask] = True
print(A)

[[ True  True  True  True]
 [False False False False]
 [ True  True  True  True]]


In [7]:
def setup():
    import torch
    import numpy as np
    X = torch.rand((1000,1000), device="cpu")
    idx_all = np.arange(0, 1000)
    idx_np = np.random.choice(idx_all, size=100, replace=False)
    idx_torch = torch.tensor(idx_np, dtype=torch.long, device="cpu")


In [9]:
import torch
import numpy as np
X = torch.rand((1000,1000), device="cpu")
idx_all = np.arange(0, 1000)
idx_np = np.random.choice(idx_all, size=100, replace=False)
idx_torch = torch.tensor(idx_np, dtype=torch.long, device="cpu")

In [10]:
%timeit X[idx_np]

25.2 µs ± 2.35 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
%timeit X[idx_torch]

18.2 µs ± 4.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
X = torch.rand((1000,1000), device="cuda:0")
idx_torch = torch.tensor(idx_np, dtype=torch.long, device="cuda:0")

In [15]:
%timeit X[idx_torch]

13.8 µs ± 113 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
%timeit X[idx_np]

46.2 µs ± 646 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
timeit.timeit("X[idx_np]", setup, number=1000)

NameError: name 'X' is not defined